In [2]:
import data_functions


In [3]:
short = data_functions.get_csv_from_txt('data/a2q50k.txt')

new


In [4]:
short

,user_from,time,user_to
0,9,2008-09-15,1217567877
1,1,2008-09-15,1217573801
2,13,2008-09-15,1217606247
3,17,2008-09-15,1217617639
4,48,2008-09-15,1217618182
...,...,...,...
49995,1856916,2008-09-15,1221948443
49996,17602,2008-09-15,1221948456
49997,19626,2008-09-15,1221948475
49998,12908,2008-09-15,1221948483


In [5]:
a2q = data_functions.get_csv_from_txt('data/sx-stackoverflow-a2q.txt')

new


In [12]:
a2q.groupby(by=['user_to', 'user_from'], as_index=False).count().sort_values(by='time', ascending=False)

,user_to,user_from,time
6109120,1340992702,250259,2
0,1217567877,9,1
11882347,1397137641,3106662,1
11882353,1397137682,1796105,1
11882352,1397137680,2503717,1
...,...,...,...
5941179,1339111992,302139,1
5941180,1339112012,1416200,1
5941181,1339112022,351385,1
5941182,1339112027,820127,1


In [20]:
a2q[(a2q.user_to==1340992702)& (a2q.user_from==250259)]

,user_from,time,user_to
6109036,250259,2008-10-02,1340992702
6109037,250259,2008-09-30,1340992702


In [6]:
c2a = data_functions.get_csv_from_txt('data/sx-stackoverflow-c2a.txt')


new


In [13]:
c2a.groupby(by=['user_to','user_from'], as_index=False).count().sort_values(by='time', ascending=False)

,user_to,user_from,time
2036962,1281821791,420211,3
20568843,1425940912,421455,3
9816906,1355450637,61654,3
3165141,1298018433,584170,3
354,1220903690,1898,3
...,...,...,...
8468416,1345735489,617450,1
8468415,1345735483,527808,1
8468414,1345735480,661519,1
8468413,1345735469,1014822,1


In [21]:
c2a.iloc[2036962].time - c2a.iloc[3165141].time

Timedelta('-3 days +00:00:00')

In [7]:
c2q = data_functions.get_csv_from_txt('data/sx-stackoverflow-c2q.txt')

new


In [206]:
import collections
def to_lists(d):
    keys = []
    vals = []
    for k,v in d.items():
        keys.append(k)
        vals.append(v)
    return [keys, vals]

# Retrieving the pairs of users that have more than one interaction
c2q_counted = c2q.groupby(by=['user_from', 'user_to'], as_index=False).count()
c2q_m1 = c2q_counted[c2q_counted.time>1]
more_than1 = c2q[(c2q.user_from.isin(c2q_m1.user_from)) & (c2q.user_to.isin(c2q_m1.user_to))]

# Grouping them by the pair and counting the results
c2q_grouped = more_than1.groupby(by=['user_from', 'user_to'], as_index=False).agg({'time': collections.Counter})

# Retrieving from the counter the columns relatives to the time and the interaction cardinality
c2q_grouped[['times', 'cardinality']]  = pd.DataFrame(c2q_grouped.time.apply(to_lists).to_list(), columns=['times','cardinality'])
c2q_grouped.drop(columns='time', inplace=True)


In [213]:
c2q_ones = c2q[~((c2q.user_from.isin(c2q_m1.user_from)) & (c2q.user_to.isin(c2q_m1.user_to)))]

In [233]:
c2q_ones.loc[:,['times', 'cardinality']]  = pd.DataFrame(c2q_ones.time.apply(lambda x: [[x], [1]]).to_list(), columns=['times','cardinality'])
c2q_ones.drop(columns='time', inplace=True)

AttributeError: 'DataFrame' object has no attribute 'time'

In [219]:
print(f"ones: {len(c2q_ones)} grouped:{len(c2q_grouped)} grouped_sum: {sum(c2q_grouped.cardinality.apply(sum))} tot: {len(c2q)} sum: {len(c2q_grouped)+len(c2q_ones)} tot_sum: {sum(c2q_grouped.cardinality.apply(sum)) + len(c2q_ones)}")

ones: 20267777 grouped:177 grouped_sum: 374 tot: 20268151 sum: 20267954 tot_sum: 20268151


In [226]:
c2q_processed = pd.concat((c2q_ones,c2q_grouped))

In [231]:
c2q_processed[c2q_processed.isna().sum(axis=1)>0]

,user_from,user_to,times,cardinality
20267777,2388760,1457265884,NaN,NaN
20267778,805548,1457265922,NaN,NaN
20267779,642070,1457265934,NaN,NaN
20267780,1815980,1457265941,NaN,NaN
20267781,5472518,1457265950,NaN,NaN
...,...,...,...,...
20268146,1177890,1457273216,NaN,NaN
20268147,3345375,1457273328,NaN,NaN
20268148,3507137,1457273391,NaN,NaN
20268149,5617035,1457273425,NaN,NaN


In [232]:
c2q[c2q.user_from==2388760]

,user_from,time,user_to
17133465,2388760,2008-10-12,1438636970
20267777,2388760,2008-10-12,1457265884


In [15]:
c2q[c2q.user_from==c2q.user_to]

,user_from,time,user_to


In [19]:
c2a[c2a.user_from==c2a.user_to]

,user_from,time,user_to


In [17]:
a2q[a2q.user_from==a2q.user_to]

,user_from,time,user_to


In [45]:
c2q.time.max()

Timestamp('2008-11-23 00:00:00')

In [51]:
import pandas as pd
from datetime import datetime
def get_csv_from_txt(fpath):
    '''
        Given the path of the txt containing the data, it returns a pandas dataframe that has 
        the dates (till the day) for each entry in format datetime.
        In this function it's added 1221436800 to the timestamp since accordingly to wikipedia en
        it's the release date of the first public beta version of the site
    '''
    print('new')
    df = pd.read_csv(fpath, header=None, sep=' ')
    df.rename(columns={0: 'user_from', 1: 'time', 2:'user_to'}, inplace=True)
    #df.time=df.time.apply(lambda t: datetime.fromtimestamp(t+1221436800))
    return df

In [52]:
c2q_bis = get_csv_from_txt('data/sx-stackoverflow-c2q.txt')

new


6024270